In [ ]:
USE DATABASE cortex_analyst_demo;
USE SCHEMA cortex_analyst_demo.revenue_timeseries;

-- Fact table: daily_revenue
CREATE OR REPLACE TABLE cortex_analyst_demo.revenue_timeseries.daily_revenue (
    date DATE,
    revenue FLOAT,
    cogs FLOAT,
    forecasted_revenue FLOAT,
    product_id INT,
    region_id INT
);

In [ ]:
USE DATABASE cortex_analyst_demo;
USE SCHEMA cortex_analyst_demo.revenue_timeseries;

-- Dimension table: product_dim
CREATE OR REPLACE TABLE cortex_analyst_demo.revenue_timeseries.product_dim (
    product_id INT,
    product_line VARCHAR(16777216)
);

In [ ]:
USE DATABASE cortex_analyst_demo;
USE SCHEMA cortex_analyst_demo.revenue_timeseries;

-- Dimension table: region_dim
CREATE OR REPLACE TABLE cortex_analyst_demo.revenue_timeseries.region_dim (
    region_id INT,
    sales_region VARCHAR(16777216),
    state VARCHAR(16777216)
);

In [ ]:
import modin.pandas as pd
import streamlit as st
# Import the Snowpark plugin for modin.
import snowflake.snowpark.modin.plugin
# Import Snowpark session
from snowflake.snowpark.context import get_active_session


In [ ]:
# Retrieve the Session
session = get_active_session()

# Set context
session.use_schema('cortex_analyst_demo.revenue_timeseries')

In [ ]:
df_d_rev = pd.read_csv('@RAW_DATA/daily_revenue.csv')
df_prod = pd.read_csv('@RAW_DATA/product.csv')
df_reg = pd.read_csv('@RAW_DATA/region.csv')

In [ ]:
st.bar_chart(df_d_rev.groupby('Region_id')['REVENUE'].sum().to_pandas())

In [ ]:
df_total_p_line_rev = df_d_rev.join(df_prod.set_index('Product_id'), 
                                    on='Product_id').groupby('Product_line')['REVENUE'].sum()
st.bar_chart(df_total_p_line_rev.to_pandas())

In [ ]:
df_d_rev_sub = df_d_rev['DATE', 'REVENUE', 'FORECASTED_REVENUE']

df_d_rev_sub['DIFF'] =  df_d_rev_sub['REVENUE'] - df_d_rev_sub['FORECASTED_REVENUE']

st.line_chart(df_d_rev_sub.head(30).to_pandas(), x='DATE', y=['DIFF','REVENUE','FORECASTED_REVENUE'], 
              color=["#FF0000", "#0000FF", "#00FF00"])

In [ ]:
# Save data to region_dim table
df_reg.to_snowflake(
    name = 'cortex_analyst_demo.revenue_timeseries.region_dim',
    if_exists='replace',
    index=False
)

# Save data to product_dim table
df_prod.to_snowflake(
    name = 'cortex_analyst_demo.revenue_timeseries.product_dim',
    if_exists='replace',
    index=False
)

df_d_rev.to_snowflake(
    name = 'cortex_analyst_demo.revenue_timeseries.daily_revenue',
    if_exists='replace',
    index=False
)


In [ ]:
select * from cortex_analyst_demo.revenue_timeseries.daily_revenue;

### Integrate Cortex Search
We will integrate Cortex Search as a way to improve literal string searches to help Cortex Analyst generate more accurate SQL queries.

In [ ]:
USE DATABASE cortex_analyst_demo;
USE SCHEMA revenue_timeseries;
use ROLE cortex_user_role;

  CREATE OR REPLACE CORTEX SEARCH SERVICE product_line_search_service
  ON product_dimension
  WAREHOUSE = cortex_analyst_wh
  TARGET_LAG = '1 hour'
  AS (
      SELECT DISTINCT product_line AS product_dimension FROM product_dim
  );

### Semantic Model Details
The semantic model file **[revenue_timeseries.yaml](https://github.com/fahadaz/cortex_analyst_hol/blob/main/data/revenue_timeseries.yaml)** is the key that unlocks Cortex Analyst's power. This YAML file dictates the tables, columns, etc. that Analyst can use in order to run queries that answer natural-language questions Let's talk a little about the details of this file:

The Semantic Model is composed of a number of different fields that help Cortex Analyst understand the specifics of your data:

- Logical Tables which are composed of Logical Columns
- Logical Columns which are one of dimensions, time_dimensions, or measures
- Relationships that exist between tables to allow for JOINS

Logical Tables are relatively straightforward- these are tables or views within a database. That's it! Pretty simple

Logical Columns get a bit more complicated; a logical column can reference an underlying physical column in a table, or it can be a expression containing one or more physical columns. So, for example, in the **[revenue_timeseries.yaml](https://github.com/fahadaz/cortex_analyst_hol/blob/main/data/revenue_timeseries.yaml)**, we have a simple logical column **daily_revenue** that is a physical column. In the **daily_revenue** measure definition, you'll notice that we provide a description, as well as synonyms, data_type, and a default_aggregation, but no **expr** parameter. This is because revenue is simply a physical column in the **daily_revenue** table:

```
measures:
    - name: daily_revenue
        expr: revenue
        description: total revenue for the given day
        synonyms: ["sales", "income"]
        default_aggregation: sum
        data_type: number
```
In contrast, we define a different measure daily_profit which is not in fact a physical column, but rather an expression of the difference between the revenue and cogs physical columns:

```
- name: daily_profit
    description: profit is the difference between revenue and expenses.
    expr: revenue - cogs
    data_type: number
```

In the semantic model, time_dimensions specifically capture temporal features of the data, and dimensions are not quantitative fields (e.g. quantitative fields are measures, while categorical fields are dimensions).

An example time_dimension:
```
time_dimensions:
    - name: date
    expr: date
    description: date with measures of revenue, COGS, and forecasted revenue for each product line
    unique: false
    data_type: date
```
An example relationship:

```
relationships:
  - name: revenue_to_product
    left_table: daily_revenue
    right_table: product
    relationship_columns:
      - left_column: product_id
        right_column: product_id
    join_type: left_outer
    relationship_type: many_to_one
```
